#### Логистическая регрессия на базе статических эмбеддингов (TFIDF+SVD, Word2Vec+SVD)

In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [2]:
from utils import *

In [3]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [5]:
def get_embedding(text, emb_dict):
    
    embs = [emb_dict[w] for w in text.split() if w in emb_dict]
    
    if len(embs) == 0: 
        first_key = next(iter(emb_dict.keys()))
        return np.zeros(emb_dict[first_key].shape)
    
    return np.vstack(embs).mean(axis=0)

* *функция для извлечения эмбеддинга документа (последовательности слов)*
* *делим предобработанный текст на слова, получаем эмбеддинг для каждого из них (из словаря)*
* *если данное слово отсутствует в словаре, пропускаем его (формируем список эмбеддингов)*
* *возвращаем вектор средних значений по каждому признаку для слов (эмбеддинг документа)*
* *в случае, если ни одно из слов не было найдено в словаре, возвращаем вектор нулевых значений*

In [ ]:
tqdm.pandas()
df = pd.read_feather('data/prep.ftr')
emb_dict = load_pickle('data/w2v_embs.bin')

ge = lambda s: get_embedding(s, emb_dict)
text_series = df['title'] + ' ' + df['text']
text_embs = text_series.progress_apply(ge)
text_embs = np.vstack(text_embs.to_numpy())

text_len = np.expand_dims(df['text_len'].to_numpy(), axis=1)
title_len = np.expand_dims(df['title_len'].to_numpy(), axis=1)

X = np.hstack((text_embs, title_len, text_len))
y = df['topic'].to_numpy()

In [7]:
text_embs.shape, X.shape, y.shape

((432158, 300), (432158, 302), (432158,))

* *загружаем предобработанный датафрейм*
* *загружаем словарь эмбеддингов слов (TFIDF или Word2Vec)*
* *склеиваем title и text для каждого из документов*
* *извлекаем эмбеддинги для каждого документа (по 300 фичей на документ)*
* *приклеиваем к кадждому из этих эмбеддингов еще по 2а признака: text_len и title_len*
* *в результате получаем 432158 примера (302 признака в каждом), а также 432158 целевых метки класса*

In [8]:
params = {'stratify': y, 'test_size': 0.2, 'random_state': 42}
X_train, X_valid, y_train, y_valid = train_test_split(X, y, **params)

* *делим данные на 2е части: 80% для обучения, стратификация по целевой переменной*

In [9]:
%%time

params = {
    'solver': 'liblinear'
}

logreg = Pipeline([
    ('sca', StandardScaler()), 
    ('clf', LogisticRegression(**params))
])

logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_valid)

CPU times: user 10min 51s, sys: 1.4 s, total: 10min 52s
Wall time: 10min 52s


* *собираем pipeline на базе логистической регрессии*
* *будем выполнять нормализацию всех 302х признаков для каждого объекта*
* *обучаем полученный pipeline, формируем предсказания на валидационной части данных*

In [12]:
accuracy_score(y_valid, y_pred)

0.925733524620511

* *на эмбеддингах из TFIDF: acc = 0.9157372269529804*
* *на эмбеддингах из Word2Vec: acc = 0.925733524620511*

In [13]:
f1_score(y_valid, y_pred, average='macro')

0.9182387016356302

* *на эмбеддингах из TFIDF: f1 = 0.9072066831050293*
* *на эмбеддингах из Word2Vec: f1 = 0.9182387016356302*

In [14]:
print(confusion_matrix(y_valid, y_pred))

[[ 7253   309   658   312    79   322]
 [  235 10076   302    77    21    48]
 [  360   236 25719   468    98   443]
 [  342   126   552  9395    12   200]
 [   58    21   114    21 12642    27]
 [  239    56   501   132    50 14928]]


* *последний запуск был на Word2Vec эмбеддингах*

In [15]:
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.81      0.83      8933
           1       0.93      0.94      0.93     10759
           2       0.92      0.94      0.93     27324
           3       0.90      0.88      0.89     10627
           4       0.98      0.98      0.98     12883
           5       0.93      0.94      0.94     15906

    accuracy                           0.93     86432
   macro avg       0.92      0.92      0.92     86432
weighted avg       0.93      0.93      0.93     86432



* *последний запуск был на Word2Vec эмбеддингах*
* *опять же, видно, что хуже всего модель справляется с классом 0 (Интернет и СМИ)*

In [16]:
list(load_json('data/labels.json').keys())

['Интернет и СМИ', 'Культура', 'Мир', 'Наука и техника', 'Спорт', 'Экономика']

In [17]:
dump_pickle(logreg, 'data/w2v_embs_logreg.bin')

* *размер файла модели: ~22k*
* *размер файла словаря эмбеддингов TFIDF: ~41Мб*
* *размер файла словаря эмбеддингов Word2Vec: ~61Мб*